In [3]:
from selenium import webdriver
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import time
import lxml
from pandas import DataFrame as df


In [4]:
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

[WDM] - Downloading: 100%|██████████| 7.46M/7.46M [00:00<00:00, 35.4MB/s]


In [47]:
driver.get('https://www.linkedin.com/uas/login?session_redirect=https%3A%2F%2Fwww%2Elinkedin%2Ecom%2Ffeed%2F&fromSignIn=true&trk=cold_join_sign_in')

username = driver.find_element("id", "username")
  
# In case of an error, try changing the element
# tag used here.
  
# Enter Your Email Address
username.send_keys("arjunkmrm@gmail.com")  
  
# entering password
pword = driver.find_element("id", "password")
# In case of an error, try changing the element 
# tag used here.
  
# Enter Your Password
pword.send_keys("arjlinZ420713~~")        
  
# Clicking on the log in button
# Format (syntax) of writing XPath --> 
# //tagname[@attribute='value']
driver.find_element("xpath", "//button[@type='submit']").click()
# In case of an error, try changing the

In [4]:
# Opening Kunal's Profile
# paste the URL of Kunal's profile here
profile_url = "https://www.linkedin.com/company/bnp-paribas/people/"
  
driver.get(profile_url)        # this will open the link

filter = driver.find_element("xpath", "//*[@id='people-search-keywords']")
#id="people-search-keywords"
filter.send_keys("sustainability")
filter.send_keys(Keys.RETURN)


In [5]:
start = time.time()
  
# will be used in the while loop
initialScroll = 0
finalScroll = 1000
  
while True:
    driver.execute_script(f"window.scrollTo({initialScroll}, {finalScroll})")
    # this command scrolls the window starting from
    # the pixel value stored in the initialScroll 
    # variable to the pixel value stored at the
    # finalScroll variable
    initialScroll = finalScroll
    finalScroll += 1000
  
    # we will stop the script for 3 seconds so that 
    # the data can load
    time.sleep(3)
    # You can change it as per your needs and internet speed
  
    end = time.time()
  
    # We will scroll for 20 seconds.
    # You can change it as per your needs and internet speed
    if round(end - start) > 120:
        break

In [6]:
src = driver.page_source
  
# Now using beautiful soup
soup = BeautifulSoup(src, 'lxml')

In [9]:


# Extracting the HTML of the complete introduction box
# that contains the name, company name, and the location
#profiles = soup.find('div', {'class': 'scaffold-finite-scroll__content'})
#print(profiles)

In [18]:
sustainability_roles_html = soup.find_all(
  'div', {'class': 'artdeco-entity-lockup__subtitle ember-view'})
sustainability_roles = []
  
for name in sustainability_roles_html:
    sustainability_roles.append(name.text.strip())
  
sustainability_roles

['Head Of Corporate Development & Sustainability, APAC - BNP Paribas',
 'Head of Sustainability Transaction Banking EMEA',
 'Corporate Development & Sustainability At BNP Paribas CIB Asia Pacific',
 'Sustainability . ESG . Impact . Asset & Wealth Management . Alternative Investments',
 'Global Head of Sustainability at BNP Paribas Asset Management',
 'Head of Corporate Social Responsibility & Sustainability, Bank of the West',
 'Chief Sustainability Officer at BNP Paribas Poland',
 'Head of Sustainability & CSR at BNP Paribas REIM',
 'Head of Sustainability Research at BNP Paribas Markets 360',
 'Head of Sustainability Communication Innovation - IRB',
 'CSR & Sustainability Manager',
 'Global Head of Sustainability',
 'Head of Sustainability and Engagement at BNP Paribas Wealth Management',
 'Head Sustainability Network at BNP Paribas Branch Brussels',
 'Global Head of Sustainability Research at BNP Paribas Asset Management',
 'Environment and Sustainability Expert',
 'Chief Sustainabi

In [34]:
def linkedin_scraper(company, role, scroll_time):
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    
    #sign in to your profile
    driver.get('https://www.linkedin.com/uas/login?session_redirect=https%3A%2F%2Fwww%2Elinkedin%2Ecom%2Ffeed%2F&fromSignIn=true&trk=cold_join_sign_in')
    #find username field
    username = driver.find_element("id", "username")
    # Enter Your Email Address
    username.send_keys("arjunkmrm@gmail.com")  
    # entering password
    pword = driver.find_element("id", "password")
    # Enter Your Password
    pword.send_keys("arjlinZ420713~~")        
    #click login button
    driver.find_element("xpath", "//button[@type='submit']").click()
    
    #go to target company
    # Opening Kunal's Profile
    # paste the URL of Kunal's profile here
    profile_url = "https://www.linkedin.com/company/" + company + "/people/"
    
    driver.get(profile_url)        # this will open the link

    filter = driver.find_element("xpath", "//*[@id='people-search-keywords']")
    #id="people-search-keywords"
    filter.send_keys("sustainability")
    filter.send_keys(Keys.RETURN)

    start = time.time()
  
    # will be used in the while loop
    initialScroll = 0
    finalScroll = 1000
    
    while True:
        driver.execute_script(f"window.scrollTo({initialScroll}, {finalScroll})")
        # this command scrolls the window starting from
        # the pixel value stored in the initialScroll 
        # variable to the pixel value stored at the
        # finalScroll variable
        initialScroll = finalScroll
        finalScroll += 1000
    
        # we will stop the script for 3 seconds so that 
        # the data can load
        time.sleep(3)
        # You can change it as per your needs and internet speed
    
        end = time.time()
    
        # We will scroll for 20 seconds.
        # You can change it as per your needs and internet speed
        if round(end - start) > scroll_time:
            break
        
    src = driver.page_source
  
    # Now using beautiful soup
    soup = BeautifulSoup(src, 'lxml')

    #sustainability related roles
    sustainability_roles_html = soup.find_all(
    'div', {'class': 'artdeco-entity-lockup__subtitle ember-view'})
    sustainability_roles = []

    #names
    sustainability_names_html = soup.find_all(
    'div', {'class': ['artdeco-entity-lockup__title ember-view']})
    sustainability_names = []
    
    for role in sustainability_roles_html:
        sustainability_roles.append(role.text.strip())

    for name in sustainability_names_html:
        sustainability_names.append(name.text.strip())

    del sustainability_names[0]
    
    sust_df = df({'name':sustainability_names, 'role':sustainability_roles})
    #names_df = df({'name':sustainability_names})
    sust_df['company'] = company

    return(sust_df)

In [35]:
scb = linkedin_scraper('standardchartered', 'sustainability', 150)
bnpp = linkedin_scraper('bnp-paribas', 'sustainability', 150)
jpm = linkedin_scraper('jpmorganchase', 'sustainability', 150)
dbs = linkedin_scraper('dbs-bank', 'sustainability', 150)



In [38]:
roles_df = df()
roles_df = pd.concat([scb, bnpp, jpm])
roles_df
#names
#scb
#'org-people-profile-card__profile-title t-black lt-line-clamp lt-line-clamp--single-line ember-view', 
#scb

,name,role,company
0,Zhi Ling Wong,"Head, Reputational & Sustainability Risk Frame...",standardchartered
1,Rahul Prem,Program Manager | Financial Markets | Sustaina...,standardchartered
2,"Alyssa Lin, CESGA",Make the world a better place.,standardchartered
3,Faisal Abdul Karim,Innovation leader in Banking | FinTech | Risk ...,standardchartered
4,Karuna Bhatia,"Head of Sustainability, Standard Chartered Ban...",standardchartered
...,...,...,...
463,LinkedIn Member,Arch. Facility Manager at JPMorgan Chase & Co....,jpmorganchase
464,PARTHA MAJUMDAR,Just a Programmer | Author | Teacher | I also ...,jpmorganchase
465,Angel Ephesus,"Company Owner at CigarSafe, LLC",jpmorganchase
466,LinkedIn Member,Environmental Risk Senior Analyst at JPMorgan ...,jpmorganchase


In [9]:
mst = linkedin_scraper('microsoft', 'sustainability', 30)
mst

,name,company
0,Microsoft,microsoft
1,Kavickumar Muruganathan,microsoft
2,Kavickumar Muruganathan,microsoft
3,Benson Ang,microsoft
4,Benson Ang,microsoft
...,...,...
113,Hyrine Munga Ph.D. SEA,microsoft
114,Hyrine Munga Ph.D. SEA,microsoft
115,Trevor Dhu,microsoft
116,Trevor Dhu,microsoft


In [ ]:
#create a network from this?

#artdeco-entity-lockup__title ember-view
